<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/StockFractionRobustness/65_Fraksi4_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data saham-saham fraksi 4 dijalankan menggunakan email shintafiaa@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



**Memasangkan Google Drive ke Google Colab**

* Kata kunci **from** digunakan untuk mengimpor hanya bagian tertentu dari modul (melakukan impor untuk bagian tertentu pada *library*)
* Modul merupakan file yang berisi sekumpulan fungsi yang ingin disertakan dalam aplikasi. Untuk membuat modul cukup simpan kode yang Anda inginkan dalam file dengan ekstensi file .py.
* **import** melakukan impor seluruh *library*.
* **Mounting** adalah proses membuat file dan direktori pada perangkat penyimpanan (seperti google drive) sehingga tersedia bagi pengguna untuk dapat diakses.



In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
    * [7.4. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)](#6.3)
      * [7.4.1. Agen 1: A2C](#6.4.1)
      * [7.4.2. Agen 2: PPO](#6.4.2)
      * [7.4.3. Agen 3: TD3](#6.4.3)
      * [7.4.4. Agen 4: SAC](#6.4.4)
      * [7.4.5. Agen 5: DDPG](#6.4.5)
    * [7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi](#6.4)
      * [7.5.1. Agen 1: A2C](#6.4.1)
      * [7.5.2. Agen 2: PPO](#6.4.2)
      * [7.5.3. Agen 3: TD3](#6.4.3)
      * [7.5.4. Agen 4: SAC](#6.4.4)
      * [7.5.5. Agen 5: DDPG](#6.4.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


* FinRL adalah *open-source framework* pertama yang menunjukkan potensi besar *Reinforcement Learning* keuangan.

* FinRL menyediakan berbagai pengaturan untuk melakukan *trading* dengan *Reinforcement Learning* seperti menyediakan ratusan pasar keuangan, algoritma yang canggih, berbagai macam aplikasi keuangan (alokasi portofolio, perdagangan mata uang kripto, *high-frequency trading*), *live trading, cloud deployment,* dll.

* Pengaplikasian pustaka FinRL dapat menggunakan alamat berikut, git+https://github.com/AI4Finance-Foundation/FinRL.git. Namun, untuk kemudahan pengaturan nilai sesuai kebutuhan, maka, dilakukanlah *forking* terhadap *repository* tersebut, lalu digunakanlah alamat berikut, git+https://github.com/shintaroudlotulhanafia/FinRL.git



In [114]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-1icze92q
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-1icze92q
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-d3ytme76/pyfolio_44e2f263d9764720b1a9f7de73fce893
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-d3ytme76/pyfolio_44e2f263d9764720b1a9f7de73fce893
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-d3ytme76/elegantrl_f5ffff64ed7f472e87e218aff3c632b3
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-d3ytme76/elegantrl_f5ffff64ed7f472e87e218aff3c632b3


<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [115]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

<a id='1.4'></a>
## 2.4. Membuat Folder

* **config** berisi pengaturan tanggal periode training dan trading, indikator, hyperparameter setiap model/agen DRL.
* **config_tickers** berisi pengaturan daftar saham yang akan diproses.
* **import os** digunakan untuk meng-import modul, merupakan module pada python agar python itu sendiri berinteraksi langsung terhadap sistem operasi.
* **check_and_make_directories** digunakan untuk mengecek dan membuat folder.


In [116]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

-----
kelas YahooDownloader:
    Mengambil data saham harian dari
    Yahoo Finance API

    Atribut
    ----------
        start_date : str
            tanggal mulai data
        end_date : str
            tanggal akhir data
        ticker_list : list
            daftar ticker saham (dimodifikasi dari config.py)

    Methods
    -------
    fetch_data()


Penetapan tanggal training dan trading dapat dilakukan dengan mengaturnya pada finrl/config.py atau dengan menetapkannya di sel notebook.

In [117]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [118]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: <Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00 - Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00 - Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00 - Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']



In [119]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi4).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10916, 8)


* **df.shape** digunakan untuk mendapatkan jumlah baris dan kolom

In [120]:
df.shape

(10916, 8)

* **nunique()** mengembalikan jumlah nilai unik untuk setiap kolom. Dengan menentukan sumbu kolom ( axis='columns' ), metode nunique() mencari berdasarkan kolom tersebut dan mengembalikan jumlah nilai unik untuk setiap baris.

In [121]:
df.nunique()

date      2729
open      2064
high      2070
low       2104
close     3386
volume    5390
tic          4
day          5
dtype: int64

* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [122]:
print(df['tic'].unique())

['JECC.JK' 'TMAS.JK' 'TPIA.JK' 'UNVR.JK']


* **describe()** digunakan untuk menampilkan detail statistik dasar seperti persentil, mean, std, dll. Hanya kolom yang bertipe numerik yang akan ditampilkan statistiknya.

In [123]:
df.describe()

,open,high,low,close,volume,day
count,10916.000000,10916.000000,10916.000000,10916.000000,1.091600e+04,10916.000000
mean,2432.774045,2456.235431,2409.271419,2088.913525,5.551627e+06,1.988641
std,2995.412968,3020.367320,2969.115061,2648.204358,1.317312e+07,1.409951
min,30.000000,30.200001,27.000000,22.313635,0.000000e+00,0.000000
25%,152.000000,154.000000,149.000000,129.375214,1.000000e+03,1.000000
50%,875.503113,903.725342,855.000000,771.150146,3.977500e+05,2.000000
75%,3790.000000,3812.500000,3760.000000,3006.314941,7.077875e+06,3.000000
max,11235.000000,11620.000000,11180.000000,9885.864258,5.311050e+08,4.000000


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan *ascending* atau *descending* dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [124]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,295.000000,295.000000,295.000000,198.915359,3,JECC.JK,0
1,2009-01-05,46.000000,47.000000,44.000000,34.214245,775000,TMAS.JK,0
2,2009-01-05,82.948853,82.948853,82.948853,66.105309,0,TPIA.JK,0
3,2009-01-05,1620.000000,1620.000000,1560.000000,1149.686401,20,UNVR.JK,0
4,2009-01-06,295.000000,295.000000,295.000000,198.915359,0,JECC.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.



-----
kelas FeatureEngineer:
    Menyediakan metode untuk preprocessing data harga saham

    Attributes
    ----------
        use_technical_indicator : boolean
            menggunakan indikator teknis atau tidak
        tech_indicator_list : list
            daftar nama indikator teknis (dimodifikasi dari neofinrl_config.py)
        use_vix : boolean
            menggunakan Volatility Index (VIX) atau tidak
        use_turbulence : boolean
            menggunakan indeks turbulensi atau tidak
        user_defined_feature:boolean
            menggunakan fitur yang ditentukan pengguna atau tidak
    Methods
    -------
    preprocess_data()
        metode utama untuk melakukan feature engineering


In [125]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [126]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [127]:
processed.describe()

,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
count,10608.000000,10608.000000,10608.000000,10608.000000,1.060800e+04,10608.000000,10608.000000,10608.000000,10608.000000,10608.000000,10608.000000
mean,2429.045974,2452.658290,2405.377227,2085.151336,5.595889e+06,2.019231,9.631031,52.085156,6.694650,18.179936,4.051032
std,2992.801240,3017.942504,2966.282618,2645.361215,1.327819e+07,1.397923,61.035822,13.003421,126.250707,7.335005,11.274543
min,30.000000,30.200001,27.000000,22.313635,0.000000e+00,0.000000,-469.537055,0.000000,-1000.000000,9.140000,0.000000
25%,152.000000,154.000000,149.000000,129.375214,1.000000e+03,1.000000,-3.242398,45.273710,-71.745486,13.287500,0.301358
50%,860.000000,890.000000,830.000000,688.192963,4.074800e+05,2.000000,0.223005,51.351767,2.236142,15.985000,1.298431
75%,3762.500000,3790.000000,3730.000000,2984.897034,7.149700e+06,3.000000,15.008079,57.991415,81.805390,20.629999,3.926853
max,11235.000000,11620.000000,11005.000000,9885.864258,5.311050e+08,4.000000,507.194433,100.000000,1000.000000,56.650002,306.792950


In [128]:
processed.nunique()

date           2652
open           2044
high           2054
low            2085
close          3332
volume         5270
tic               4
day               5
macd          10592
rsi_30         7195
cci_30        10173
vix            1441
turbulence     2398
dtype: int64

* **tolist()** digunakan untuk mengubah elemen data array menjadi *list*.
* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [129]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

* **date_range()** digunakan untuk mendapatkan frekuensi tetap DatetimeIndex. date_range() digunakan untuk membuat rentang tanggal di pandas.
* **max()** mengembalikan item dengan nilai tertinggi, atau item dengan nilai tertinggi dalam *iterable*. Jika nilainya adalah string, maka dilakukan perbandingan abjad.
* **min()** mengembalikan item dengan nilai terendah, atau item dengan nilai terendah dalam iterable. Jika nilainya adalah string,  maka dilakukan perbandingan abjad.
* **astype()** digunakan untuk mengubah tipe data dari suatu bentuk *series*.

In [130]:
list_date = list(pd.date_range(df['date'].min(),df['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

* **Array** dapat menyimpan elemen hanya dari satu tipe. Sedangkan **list** juga dapat menyimpan elemen dari tipe data yang berbeda.
* **list()** digunakan untuk membuat objek *list*. Objek *list* adalah kumpulan yang berurutan dan dapat diubah.
* **itertools.product()** digunakan untuk mencari produk kartesius dari iterator yang diberikan, outputnya adalah urutan leksikografis.

In [131]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'JECC.JK'),
 ('2009-01-05', 'TMAS.JK'),
 ('2009-01-05', 'TPIA.JK'),
 ('2009-01-05', 'UNVR.JK'),
 ('2009-01-06', 'JECC.JK'),
 ('2009-01-06', 'TMAS.JK'),
 ('2009-01-06', 'TPIA.JK'),
 ('2009-01-06', 'UNVR.JK'),
 ('2009-01-07', 'JECC.JK'),
 ('2009-01-07', 'TMAS.JK'),
 ('2009-01-07', 'TPIA.JK'),
 ('2009-01-07', 'UNVR.JK'),
 ('2009-01-08', 'JECC.JK'),
 ('2009-01-08', 'TMAS.JK'),
 ('2009-01-08', 'TPIA.JK'),
 ('2009-01-08', 'UNVR.JK'),
 ('2009-01-09', 'JECC.JK'),
 ('2009-01-09', 'TMAS.JK'),
 ('2009-01-09', 'TPIA.JK'),
 ('2009-01-09', 'UNVR.JK'),
 ('2009-01-10', 'JECC.JK'),
 ('2009-01-10', 'TMAS.JK'),
 ('2009-01-10', 'TPIA.JK'),
 ('2009-01-10', 'UNVR.JK'),
 ('2009-01-11', 'JECC.JK'),
 ('2009-01-11', 'TMAS.JK'),
 ('2009-01-11', 'TPIA.JK'),
 ('2009-01-11', 'UNVR.JK'),
 ('2009-01-12', 'JECC.JK'),
 ('2009-01-12', 'TMAS.JK'),
 ('2009-01-12', 'TPIA.JK'),
 ('2009-01-12', 'UNVR.JK'),
 ('2009-01-13', 'JECC.JK'),
 ('2009-01-13', 'TMAS.JK'),
 ('2009-01-13', 'TPIA.JK'),
 ('2009-01-13', 'UNV

* **Pandas DataFrame** adalah dua dimensi yang dapat berubah ukuran, struktur data tabular yang berpotensi heterogen dengan sumbu berlabel (baris dan kolom).
* **merge()** berfungsi untuk memperbarui konten dua DataFrame dengan menggabungkannya bersama-sama, menggunakan metode yang ditentukan.

In [132]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915359,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105309,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915359,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15999,2019-12-18,UNVR.JK,8195.000000,8205.000000,8120.000000,7602.727051,15872500.0,2.0,-74.938459,41.772053,-169.670759,12.580000,0.112174
16000,2019-12-19,JECC.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16001,2019-12-19,TMAS.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16002,2019-12-19,TPIA.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* **isin()** digunakan untuk memfilter data frames. isin() dapat membantu untuk memilih baris dengan memiliki nilai tertentu (atau beberapa nilai tertentu) dalam kolom tertentu.

In [133]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915359,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105309,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915359,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,8180.000000,8225.000000,8180.000000,7612.015625,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.290000,0.019866
15996,2019-12-18,JECC.JK,6175.000000,6175.000000,6175.000000,6175.000000,0.0,2.0,-2.741289,53.058014,-164.670433,12.580000,0.112174
15997,2019-12-18,TMAS.JK,98.000000,100.000000,98.000000,80.050919,2135700.0,2.0,-3.212769,38.307125,-74.912962,12.580000,0.112174
15998,2019-12-18,TPIA.JK,2487.500000,2500.000000,2481.250000,2471.146240,22134800.0,2.0,45.007717,64.526587,113.802417,12.580000,0.112174


In [134]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915359,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105309,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915359,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,8180.000000,8225.000000,8180.000000,7612.015625,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.290000,0.019866
15996,2019-12-18,JECC.JK,6175.000000,6175.000000,6175.000000,6175.000000,0.0,2.0,-2.741289,53.058014,-164.670433,12.580000,0.112174
15997,2019-12-18,TMAS.JK,98.000000,100.000000,98.000000,80.050919,2135700.0,2.0,-3.212769,38.307125,-74.912962,12.580000,0.112174
15998,2019-12-18,TPIA.JK,2487.500000,2500.000000,2481.250000,2471.146240,22134800.0,2.0,45.007717,64.526587,113.802417,12.580000,0.112174


* **fillna()** digunakan untuk mengisi nilai NA/NaN menggunakan metode yang ditentukan. fillna(0) artinya baris NA/NaN diisi dengan nilai misalnya 0.

In [135]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915359,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105309,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915359,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,8180.000000,8225.000000,8180.000000,7612.015625,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.290000,0.019866
15996,2019-12-18,JECC.JK,6175.000000,6175.000000,6175.000000,6175.000000,0.0,2.0,-2.741289,53.058014,-164.670433,12.580000,0.112174
15997,2019-12-18,TMAS.JK,98.000000,100.000000,98.000000,80.050919,2135700.0,2.0,-3.212769,38.307125,-74.912962,12.580000,0.112174
15998,2019-12-18,TPIA.JK,2487.500000,2500.000000,2481.250000,2471.146240,22134800.0,2.0,45.007717,64.526587,113.802417,12.580000,0.112174


In [136]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915359,3.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105309,0.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915359,0.0,1.0,0.000000,0.0,-66.666667,38.560001,0.0
5,2009-01-06,TMAS.JK,46.000000,47.000000,44.000000,33.470455,1107500.0,1.0,-0.016688,0.0,-66.666667,38.560001,0.0
6,2009-01-06,TPIA.JK,82.948853,82.948853,82.948853,66.105309,0.0,1.0,0.000000,0.0,-66.666667,38.560001,0.0
7,2009-01-06,UNVR.JK,1620.000000,1670.000000,1600.000000,1163.880127,16432500.0,1.0,0.318449,100.0,66.666667,38.560001,0.0
8,2009-01-07,JECC.JK,270.000000,270.000000,270.000000,182.058121,10000.0,2.0,-0.522742,0.0,-100.000000,43.389999,0.0
9,2009-01-07,TMAS.JK,44.000000,48.000000,44.000000,32.726669,1765000.0,2.0,-0.044328,0.0,-18.789015,43.389999,0.0


In [137]:
processed_full.nunique()

date           2652
tic               4
open           2044
high           2054
low            2085
close          3332
volume         5270
day               5
macd          10592
rsi_30         7195
cci_30        10173
vix            1441
turbulence     2398
dtype: int64

* **info()** berfungsi untuk mencetak informasi tentang DataFrame. Informasi tersebut berisi jumlah kolom, label kolom, tipe data kolom, penggunaan memori, indeks rentang, dan jumlah sel di setiap kolom (nilai bukan nol).

In [138]:
processed_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10608 entries, 0 to 15999
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        10608 non-null  object 
 1   tic         10608 non-null  object 
 2   open        10608 non-null  float64
 3   high        10608 non-null  float64
 4   low         10608 non-null  float64
 5   close       10608 non-null  float64
 6   volume      10608 non-null  float64
 7   day         10608 non-null  float64
 8   macd        10608 non-null  float64
 9   rsi_30      10608 non-null  float64
 10  cci_30      10608 non-null  float64
 11  vix         10608 non-null  float64
 12  turbulence  10608 non-null  float64
dtypes: float64(11), object(2)
memory usage: 1.1+ MB


* **to_csv()** mengonversi DataFrame menjadi data CSV/comma separated value (nilai yang dipisahkan koma). Kita dapat melewatkan objek file untuk menulis data CSV ke dalam file. Jika tidak, data CSV dikembalikan dalam format string.
* **loc** berbasis label, yang berarti baris dan kolom diambil berdasarkan label baris dan kolomnya. **iloc** berbasis posisi integer, maka baris dan kolom dapat diambil dengan menggunakan nilai posisi integernya (posisi integer berbasis 0).

In [139]:
#Save the dataset by the tic filter
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  df_temp.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/DatasetsPerTic/Fraksi4/'+ stock +'.csv',index=False)
  display(df_temp)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.0,295.0,295.0,198.915359,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.0,295.0,295.0,198.915359,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
8,2009-01-07,JECC.JK,270.0,270.0,270.0,182.058121,10000.0,2.0,-0.522742,0.000000,-100.000000,43.389999,0.000000
12,2009-01-08,JECC.JK,250.0,250.0,250.0,168.572327,37500.0,3.0,-1.234584,0.000000,-104.761906,42.560001,0.000000
16,2009-01-09,JECC.JK,250.0,250.0,250.0,168.572327,0.0,4.0,-1.595397,0.000000,-79.710146,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,3.0,-3.619868,53.058014,-40.969126,13.940000,3.749016
15976,2019-12-13,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,4.0,-3.380631,53.058014,-139.322722,12.630000,0.593754
15988,2019-12-16,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,0.0,-3.154669,53.058014,-151.408653,12.140000,0.376881
15992,2019-12-17,JECC.JK,6175.0,6175.0,6175.0,6175.000000,0.0,1.0,-2.941682,53.058014,-167.379002,12.290000,0.019866


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
1,2009-01-05,TMAS.JK,46.0,47.0,44.0,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
5,2009-01-06,TMAS.JK,46.0,47.0,44.0,33.470455,1107500.0,1.0,-0.016688,0.000000,-66.666667,38.560001,0.000000
9,2009-01-07,TMAS.JK,44.0,48.0,44.0,32.726669,1765000.0,2.0,-0.044328,0.000000,-18.789015,43.389999,0.000000
13,2009-01-08,TMAS.JK,42.0,45.0,42.0,31.239092,330000.0,3.0,-0.109522,0.000000,-133.333333,42.560001,0.000000
17,2009-01-09,TMAS.JK,44.0,44.0,42.0,32.726669,1590000.0,4.0,-0.083447,34.655609,-77.976581,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,99.0,100.0,97.0,78.433723,3130800.0,3.0,-3.753757,35.769414,-110.875785,13.940000,3.749016
15977,2019-12-13,TMAS.JK,98.0,100.0,98.0,80.050919,1143700.0,4.0,-3.607022,37.844103,-93.126697,12.630000,0.593754
15989,2019-12-16,TMAS.JK,100.0,100.0,98.0,79.242317,1726300.0,0.0,-3.515456,37.222220,-89.104445,12.140000,0.376881
15993,2019-12-17,TMAS.JK,99.0,99.0,97.0,79.242317,2965500.0,1.0,-3.403655,37.222220,-89.887150,12.290000,0.019866


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105309,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
6,2009-01-06,TPIA.JK,82.948853,82.948853,82.948853,66.105309,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
10,2009-01-07,TPIA.JK,82.948853,82.948853,82.948853,66.105309,61.0,2.0,0.000000,0.000000,-18.789015,43.389999,0.000000
14,2009-01-08,TPIA.JK,82.948853,82.948853,82.948853,66.105309,0.0,3.0,0.000000,0.000000,-133.333333,42.560001,0.000000
18,2009-01-09,TPIA.JK,82.948853,82.948853,82.948853,66.105309,0.0,4.0,0.000000,34.655609,-77.976581,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,2462.500000,2487.500000,2450.000000,2440.179688,32832800.0,3.0,37.659415,62.950311,136.531560,13.940000,3.749016
15978,2019-12-13,TPIA.JK,2468.750000,2487.500000,2462.500000,2452.566406,25854400.0,4.0,40.175702,63.571502,132.220615,12.630000,0.593754
15990,2019-12-16,TPIA.JK,2475.000000,2493.750000,2462.500000,2458.759766,20370000.0,0.0,42.183367,63.884705,123.628289,12.140000,0.376881
15994,2019-12-17,TPIA.JK,2481.250000,2506.250000,2475.000000,2464.953125,20856400.0,1.0,43.769658,64.203090,122.159660,12.290000,0.019866


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
3,2009-01-05,UNVR.JK,1620.0,1620.0,1560.0,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
7,2009-01-06,UNVR.JK,1620.0,1670.0,1600.0,1163.880127,16432500.0,1.0,0.318449,100.000000,66.666667,38.560001,0.000000
11,2009-01-07,UNVR.JK,1560.0,1660.0,1550.0,1107.105713,20.0,2.0,-1.354804,19.463222,-58.540160,43.389999,0.000000
15,2009-01-08,UNVR.JK,1590.0,1610.0,1570.0,1128.395874,20.0,3.0,-1.313956,38.635072,-59.838763,42.560001,0.000000
19,2009-01-09,UNVR.JK,1600.0,1630.0,1580.0,1135.492554,20.0,4.0,-0.981705,43.290158,-2.720503,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,8350.0,8420.0,8200.0,7616.660156,13365500.0,3.0,-49.981812,41.652655,-139.322722,13.940000,3.749016
15979,2019-12-13,UNVR.JK,8280.0,8300.0,8200.0,7658.458984,12891500.0,4.0,-56.350281,42.737188,-151.408653,12.630000,0.593754
15991,2019-12-16,UNVR.JK,8250.0,8275.0,8180.0,7598.083008,12503000.0,0.0,-65.513971,41.582265,-167.379002,12.140000,0.376881
15995,2019-12-17,UNVR.JK,8180.0,8225.0,8180.0,7612.015625,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.290000,0.019866


In [140]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,29500.000000,29500.000000,29500.000000,19891.535950,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,4600.000000,4700.000000,4400.000000,3421.424484,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,8294.885254,8294.885254,8294.885254,6610.530853,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,162000.000000,162000.000000,156000.000000,114968.640137,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,29500.000000,29500.000000,29500.000000,19891.535950,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,818000.000000,822500.000000,818000.000000,761201.562500,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.290000,0.019866
15996,2019-12-18,JECC.JK,617500.000000,617500.000000,617500.000000,617500.000000,0.0,2.0,-2.741289,53.058014,-164.670433,12.580000,0.112174
15997,2019-12-18,TMAS.JK,9800.000000,10000.000000,9800.000000,8005.091858,2135700.0,2.0,-3.212769,38.307125,-74.912962,12.580000,0.112174
15998,2019-12-18,TPIA.JK,248750.000000,250000.000000,248125.000000,247114.624023,22134800.0,2.0,45.007717,64.526587,113.802417,12.580000,0.112174


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

* **data_split** membagi data menjadi himpunan *train, test,* dan/atau *validation.*
* **len()** mengembalikan jumlah item dalam suatu objek. Namun, jika objek adalah string, maka fungsi len() akan mengembalikan jumlah karakter dalam string.
* **tail()** digunakan untuk mendapatkan n baris terakhir.

In [141]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1980


In [142]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,UNVR.JK,1083000.0,1088000.0,1061000.0,962058.984375,14564500.0,3.0,222.473323,67.975755,120.659931,10.18,5.395486
2156,2017-12-29,JECC.JK,470000.0,470000.0,470000.0,434712.109375,100.0,4.0,-45.023809,43.469840,-56.271744,11.04,15.259303
2156,2017-12-29,TMAS.JK,22800.0,22900.0,22800.0,18516.827393,3500.0,4.0,-1.871566,48.538842,-32.664587,11.04,15.259303
2156,2017-12-29,TPIA.JK,143125.0,150000.0,141875.0,146239.038086,24246800.0,4.0,12.320739,63.029740,167.268783,11.04,15.259303
2156,2017-12-29,UNVR.JK,1088000.0,1119500.0,1087500.0,988586.425781,12763000.0,4.0,243.419083,72.053431,162.612830,11.04,15.259303


In [143]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,JECC.JK,470000.0,470000.0,470000.0,434712.109375,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062
0,2018-01-02,TMAS.JK,22900.0,22900.0,21800.0,17627.372742,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062
0,2018-01-02,TPIA.JK,150625.0,150625.0,143750.0,144411.059570,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062
0,2018-01-02,UNVR.JK,1123500.0,1162000.0,1100000.0,988144.335938,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062
1,2018-01-03,JECC.JK,470000.0,470000.0,470000.0,434712.109375,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855


In [144]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


Transaction Fee
    # BUY Fee => 0.23% (Broker Fee(0.19%) + Levy(0.04%))
    # SELL Fee => 0.54% (Broker Fee(0.29%) + Levy(0.04%) + PPN(0.11%) + PPh(0.1%))

In [145]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [146]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.



-----
kelas DRLAgent:
    Menyediakan implementasi untuk algoritma DRL

    Atribut
    ----------
        env: gym environment class
            kelas yang ditentukan pengguna

    Methods
    -------
        get_model()
            mengatur algoritma DRL
        train_model()
            melatih algoritma DRL dalam set data latih dan mengeluarkan model yang dilatih
        DRL_prediction()
            membuat prediksi dalam kumpulan data pengujian dan dapatkan hasil


In [147]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.1'></a>
##6.1. Agen 1: A2C


In [148]:
A2C_PARAMS = {
    "n_steps": 150, 
    "ent_coef": 0.0001, 
    "learning_rate": 0.0014,
}
model_a2c = agent.get_model("a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 150, 'ent_coef': 0.0001, 'learning_rate': 0.0014}
Using cuda device


In [149]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 255       |
|    iterations         | 100       |
|    time_elapsed       | 58        |
|    total_timesteps    | 15000     |
| train/                |           |
|    entropy_loss       | -5.71     |
|    explained_variance | -0.000639 |
|    learning_rate      | 0.0014    |
|    n_updates          | 99        |
|    policy_loss        | -6.67e+03 |
|    reward             | 0.0       |
|    std                | 1.01      |
|    value_loss         | 1.52e+06  |
-------------------------------------
day: 2156, episode: 10
begin_total_asset: 100000000.00
end_total_asset: 114145502.48
total_reward: 14145502.48
total_cost: 123845938.96
total_trades: 6097
Sharpe: 0.210
-------------------------------------
| time/                 |           |
|    fps                | 259       |
|    iterations         | 200       |
|    time_elapsed       | 115       |
|    total_timesteps    | 30000  

<a id='5.2'></a>
##6.2. Agen 2: PPO

In [150]:
PPO_PARAMS = {
    "n_steps": 50000,
    "ent_coef": 0.7,
    "learning_rate": 0.0016,
    "batch_size": 1750,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 50000, 'ent_coef': 0.7, 'learning_rate': 0.0016, 'batch_size': 1750}
Using cuda device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:147: UserWarning: You have specified a mini-batch size of 1750, but because the `RolloutBuffer` is of size `n_steps * n_envs = 50000`, after every 28 untruncated mini-batches, there will be a truncated mini-batch of size 1000
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=50000 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [151]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 2156, episode: 30
begin_total_asset: 100000000.00
end_total_asset: 65162396.97
total_reward: -34837603.03
total_cost: 195456488.72
total_trades: 7855
Sharpe: -0.090
day: 2156, episode: 40
begin_total_asset: 100000000.00
end_total_asset: 198303974.37
total_reward: 98303974.37
total_cost: 254859457.78
total_trades: 8093
Sharpe: 0.429
----------------------------------
| time/              |           |
|    fps             | 270       |
|    iterations      | 1         |
|    time_elapsed    | 185       |
|    total_timesteps | 50000     |
| train/             |           |
|    reward          | 72.682526 |
----------------------------------


<a id='5.3'></a>
##6.3. Agen 3: TD3

In [152]:
TD3_PARAMS = {
    "batch_size": 7000, 
    "buffer_size": 10000, 
    "learning_rate": 0.9  
}
model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 7000, 'buffer_size': 10000, 'learning_rate': 0.9}
Using cuda device


In [153]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000)

day: 2156, episode: 50
begin_total_asset: 100000000.00
end_total_asset: 2172260847.41
total_reward: 2072260847.41
total_cost: 22639346.56
total_trades: 4435
Sharpe: 1.360
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 79        |
|    time_elapsed    | 108       |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 2.14e+07  |
|    critic_loss     | 1.17e+17  |
|    learning_rate   | 0.9       |
|    n_updates       | 6471      |
|    reward          | 1390.7354 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 69        |
|    time_elapsed    | 246       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 3.81e+10  |
|    critic_loss     | 1.05e+22  |
|    learning_rate   | 0.9       |
|    n_updates       | 1

<a id='5.4'></a>
##6.4. Agen 4: SAC

In [154]:
SAC_PARAMS = {
    "ent_coef": 1.25,
    "learning_rate": 0.0001,
    "batch_size": 4000,
    "buffer_size": 5000,
    "learning_starts": 14,
    
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'ent_coef': 1.25, 'learning_rate': 0.0001, 'batch_size': 4000, 'buffer_size': 5000, 'learning_starts': 14}
Using cuda device


In [157]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 48       |
|    time_elapsed    | 178      |
|    total_timesteps | 8628     |
| train/             |          |
|    actor_loss      | 1.5e+04  |
|    critic_loss     | 2.25e+05 |
|    ent_coef        | 1.25     |
|    learning_rate   | 0.0001   |
|    n_updates       | 29019    |
|    reward          | 271.124  |
---------------------------------
day: 2156, episode: 90
begin_total_asset: 100000000.00
end_total_asset: 100065609.16
total_reward: 65609.16
total_cost: 229472.17
total_trades: 8624
Sharpe: 0.452
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 48       |
|    time_elapsed    | 356      |
|    total_timesteps | 17256    |
| train/             |          |
|    actor_loss      | 1.48e+04 |
|    critic_loss     | 1.24e+05 |
|    ent_coef        | 1.25     |
|    learning_rate   

<a id='5.5'></a>
##6.5. Agen 5: DDPG

In [158]:
DDPG_PARAMS = {
    "batch_size": 1250, 
    "buffer_size": 50000, 
    "learning_rate": 0.0003
}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 1250, 'buffer_size': 50000, 'learning_rate': 0.0003}
Using cuda device


In [159]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=60000)

day: 2156, episode: 100
begin_total_asset: 100000000.00
end_total_asset: 1597880207.21
total_reward: 1497880207.21
total_cost: 3593118.30
total_trades: 4567
Sharpe: 0.963
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 89       |
|    time_elapsed    | 96       |
|    total_timesteps | 8628     |
| train/             |          |
|    actor_loss      | 1.9e+04  |
|    critic_loss     | 2.53e+08 |
|    learning_rate   | 0.0003   |
|    n_updates       | 6471     |
|    reward          | 3345.954 |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 79       |
|    time_elapsed    | 216      |
|    total_timesteps | 17256    |
| train/             |          |
|    actor_loss      | 1.18e+03 |
|    critic_loss     | 1.26e+07 |
|    learning_rate   | 0.0003   |
|    n_updates       | 15099    |
|    reward  

<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp10.000.000.

Pengaturan ambang turbulensi *(turbulence threshold)*

Atur ambang turbulensi menjadi lebih besar dari maksimum dalam data turbulensi sampel. Jika indeks turbulensi saat ini lebih besar dari ambang batas, maka dapat diasumsikan bahwa pasar pada periode tersebut sedang bergejolak

* **drop_duplicates()** menghapus baris duplikat.
* **quantile()** digunakan untuk menghitung kuantil nilai dalam sumbu tertentu. Sumbu default adalah baris.

In [ ]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [ ]:
insample_risk_indicator.vix.describe()

In [162]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [163]:
insample_risk_indicator.turbulence.describe()

count    2157.000000
mean        3.855361
std        11.346042
min         0.000000
25%         0.269252
50%         1.242770
75%         3.754822
max       306.792950
Name: turbulence, dtype: float64

In [164]:
insample_risk_indicator.turbulence.quantile(0.996)

50.30346173550309

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [165]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [166]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,JECC.JK,470000.0,470000.0,470000.0,434712.109375,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062
0,2018-01-02,TMAS.JK,22900.0,22900.0,21800.0,17627.372742,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062
0,2018-01-02,TPIA.JK,150625.0,150625.0,143750.0,144411.059570,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062
0,2018-01-02,UNVR.JK,1123500.0,1162000.0,1100000.0,988144.335938,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062
1,2018-01-03,JECC.JK,470000.0,470000.0,470000.0,434712.109375,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855


<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

<a id='6.3.1'></a>
###7.3.1. Agen 1: A2C

In [167]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


In [168]:
df_actions_and_account_value_a2c = pd.merge(df_actions_a2c, df_account_value_a2c, on='date', how='outer')

#adding new column
df_actions_and_account_value_a2c['total_shares'] = ""

#count the number of column of tics
numberOfColumns = len(df_actions_and_account_value_a2c.columns)
numberOfTic = numberOfColumns-2

#selecting column of actions in every tics
action_columns_a2c = df_actions_and_account_value_a2c.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_a2c['total_shares'] = action_columns_a2c.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_a2c['total_shares'] = df_actions_and_account_value_a2c['total_shares'].cumsum(skipna=False)

In [169]:
#define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

#swap points and rebounds columns
df_actions_and_account_value_a2c = swap_columns(df_actions_and_account_value_a2c, 'total_shares','account_value')

#Replace NaN value to 0
df_actions_and_account_value_a2c=df_actions_and_account_value_a2c.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_a2c)

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,67.0,0.0,0.0,0.0,67.0,1.000000e+08
1,2018-01-03,67.0,0.0,0.0,0.0,134.0,9.993301e+07
2,2018-01-04,67.0,0.0,0.0,0.0,201.0,9.986602e+07
3,2018-01-05,28.0,0.0,0.0,0.0,229.0,9.979903e+07
4,2018-01-08,0.0,11.0,0.0,0.0,240.0,1.098319e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,241.0,1.415029e+08
491,2019-12-13,0.0,0.0,0.0,0.0,241.0,1.415048e+08
492,2019-12-16,0.0,0.0,0.0,0.0,241.0,1.415039e+08
493,2019-12-17,0.0,0.0,0.0,0.0,241.0,1.415039e+08


In [170]:
df_actions_and_account_value_a2c.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi4/df_actions_and_account_value_a2c(4).csv',index=False)

<a id='6.3.2'></a>
###7.3.2. Agen 2: PPO

In [171]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


In [172]:
df_actions_and_account_value_ppo = pd.merge(df_actions_ppo, df_account_value_ppo, on='date', how='outer')

#adding new column
df_actions_and_account_value_ppo['total_shares'] = ""

#selecting column of actions in every tics
action_columns_ppo = df_actions_and_account_value_ppo.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_ppo['total_shares'] = action_columns_ppo.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_ppo['total_shares'] = df_actions_and_account_value_ppo['total_shares'].cumsum(skipna=False)

In [173]:
#swap points and rebounds columns
df_actions_and_account_value_ppo = swap_columns(df_actions_and_account_value_ppo, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_ppo=df_actions_and_account_value_ppo.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_ppo)

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,0.0,0.0,8.0,4.0,12.0,1.000000e+08
1,2018-01-03,0.0,0.0,8.0,4.0,24.0,9.982637e+07
2,2018-01-04,0.0,0.0,8.0,4.0,36.0,9.964236e+07
3,2018-01-05,0.0,0.0,8.0,4.0,48.0,9.984626e+07
4,2018-01-08,0.0,0.0,8.0,4.0,60.0,9.998710e+07
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,233.0,9.836239e+07
491,2019-12-13,0.0,0.0,0.0,0.0,233.0,9.888787e+07
492,2019-12-16,0.0,0.0,0.0,0.0,233.0,9.848351e+07
493,2019-12-17,0.0,0.0,0.0,0.0,233.0,9.868880e+07


In [174]:
df_actions_and_account_value_ppo.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi4/df_actions_and_account_value_ppo(4).csv',index=False)

<a id='6.3.3'></a>
###7.3.3. Agen 3: TD3

In [175]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym)

hit end!


In [176]:
df_actions_and_account_value_td3 = pd.merge(df_actions_td3, df_account_value_td3, on='date', how='outer')

#adding new column
df_actions_and_account_value_td3['total_shares'] = ""

#selecting column of actions in every tics
action_columns_td3 = df_actions_and_account_value_td3.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_td3['total_shares'] = action_columns_td3.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_td3['total_shares'] = df_actions_and_account_value_td3['total_shares'].cumsum(skipna=False)

In [177]:
#swap points and rebounds columns
df_actions_and_account_value_td3 = swap_columns(df_actions_and_account_value_td3, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_td3=df_actions_and_account_value_td3.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_td3)

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,100.0,0.0,100.0,0.0,200.0,1.000000e+08
1,2018-01-03,63.0,0.0,100.0,0.0,363.0,9.950120e+07
2,2018-01-04,0.0,0.0,2.0,0.0,365.0,9.830904e+07
3,2018-01-05,0.0,0.0,0.0,0.0,365.0,9.904693e+07
4,2018-01-08,0.0,0.0,0.0,0.0,365.0,1.064543e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,366.0,1.501889e+08
491,2019-12-13,0.0,0.0,0.0,0.0,366.0,1.504404e+08
492,2019-12-16,0.0,0.0,0.0,0.0,366.0,1.505661e+08
493,2019-12-17,0.0,0.0,0.0,0.0,366.0,1.506918e+08


In [178]:
df_actions_and_account_value_td3.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi4/df_actions_and_account_value_td3(4).csv',index=False)

<a id='6.3.4'></a>
###7.3.4. Agen 4: SAC

In [179]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


In [180]:
df_actions_and_account_value_sac = pd.merge(df_actions_sac, df_account_value_sac, on='date', how='outer')

#adding new column
df_actions_and_account_value_sac['total_shares'] = ""

#selecting column of actions in every tics
action_columns_sac = df_actions_and_account_value_sac.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_sac['total_shares'] = action_columns_sac.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_sac['total_shares'] = df_actions_and_account_value_sac['total_shares'].cumsum(skipna=False)

In [181]:
#swap points and rebounds columns
df_actions_and_account_value_sac = swap_columns(df_actions_and_account_value_sac, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_sac=df_actions_and_account_value_sac.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_sac)

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,0.0,5.0,6.0,100.0,111.0,1.000000e+08
1,2018-01-03,0.0,0.0,0.0,0.0,111.0,9.643914e+07
2,2018-01-04,0.0,0.0,0.0,0.0,111.0,9.534190e+07
3,2018-01-05,0.0,0.0,0.0,0.0,111.0,9.642655e+07
4,2018-01-08,0.0,0.0,0.0,0.0,111.0,9.714247e+07
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,111.0,7.767142e+07
491,2019-12-13,0.0,0.0,0.0,0.0,111.0,7.809765e+07
492,2019-12-16,0.0,0.0,0.0,0.0,111.0,7.749720e+07
493,2019-12-17,0.0,0.0,0.0,0.0,111.0,7.764024e+07


In [182]:
df_actions_and_account_value_sac.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi4/df_actions_and_account_value_sac(3).csv',index=False)

<a id='6.3.5'></a>
###7.3.5. Agen 5: DDPG

In [183]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

hit end!


In [184]:
df_actions_and_account_value_ddpg = pd.merge(df_actions_ddpg, df_account_value_ddpg, on='date', how='outer')

#adding new column
df_actions_and_account_value_ddpg['total_shares'] = ""

#selecting column of actions in every tics
action_columns_ddpg = df_actions_and_account_value_ddpg.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_ddpg['total_shares'] = action_columns_ddpg.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_ddpg['total_shares'] = df_actions_and_account_value_ddpg['total_shares'].cumsum(skipna=False)

In [185]:
#swap points and rebounds columns
df_actions_and_account_value_ddpg = swap_columns(df_actions_and_account_value_ddpg, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_ddpg=df_actions_and_account_value_ddpg.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_ddpg)

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,100.0,100.0,100.0,0.0,300.0,1.000000e+08
1,2018-01-03,55.0,100.0,100.0,0.0,555.0,9.962652e+07
2,2018-01-04,-100.0,100.0,100.0,0.0,655.0,9.830864e+07
3,2018-01-05,28.0,100.0,100.0,0.0,883.0,9.923240e+07
4,2018-01-08,-83.0,100.0,100.0,0.0,1000.0,1.033992e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,1262.0,1.645967e+08
491,2019-12-13,0.0,0.0,0.0,0.0,1262.0,1.655062e+08
492,2019-12-16,0.0,0.0,0.0,0.0,1262.0,1.658628e+08
493,2019-12-17,0.0,0.0,0.0,0.0,1262.0,1.662684e+08


In [186]:
df_actions_and_account_value_ddpg.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi4/df_actions_and_account_value_ddpg(4).csv',index=False)

##7.4 Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)

###7.4.1. Agen 1: A2C

In [187]:
#Replace NaN value to 0
action_columns_a2c=action_columns_a2c.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_a2c)

j=0
for j in range(numberOfRows):
  for i in action_columns_a2c:
   val = action_columns_a2c[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_a2c, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi4/df_actions_and_account_str_value_a2c(4).csv',index=False)
df_actions_str

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,Beli 67.0 saham,Tahan,Tahan,Tahan,67.0,1.000000e+08
1,2018-01-03,Beli 67.0 saham,Tahan,Tahan,Tahan,134.0,9.993301e+07
2,2018-01-04,Beli 67.0 saham,Tahan,Tahan,Tahan,201.0,9.986602e+07
3,2018-01-05,Beli 28.0 saham,Tahan,Tahan,Tahan,229.0,9.979903e+07
4,2018-01-08,Tahan,Beli 11.0 saham,Tahan,Tahan,240.0,1.098319e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,241.0,1.415029e+08
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,241.0,1.415048e+08
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,241.0,1.415039e+08
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,241.0,1.415039e+08


###7.4.2. Agen 2: PPO

In [188]:
#Replace NaN value to 0
action_columns_ppo=action_columns_ppo.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_ppo)

j=0
for j in range(numberOfRows):
  for i in action_columns_ppo:
   val = action_columns_ppo[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_ppo, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi4/df_actions_and_account_str_value_ppo(4).csv',index=False)
df_actions_str

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,Tahan,Tahan,Beli 8.0 saham,Beli 4.0 saham,12.0,1.000000e+08
1,2018-01-03,Tahan,Tahan,Beli 8.0 saham,Beli 4.0 saham,24.0,9.982637e+07
2,2018-01-04,Tahan,Tahan,Beli 8.0 saham,Beli 4.0 saham,36.0,9.964236e+07
3,2018-01-05,Tahan,Tahan,Beli 8.0 saham,Beli 4.0 saham,48.0,9.984626e+07
4,2018-01-08,Tahan,Tahan,Beli 8.0 saham,Beli 4.0 saham,60.0,9.998710e+07
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,233.0,9.836239e+07
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,233.0,9.888787e+07
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,233.0,9.848351e+07
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,233.0,9.868880e+07


###7.4.1. Agen 3: TD3

In [189]:
#Replace NaN value to 0
action_columns_td3=action_columns_td3.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_td3)

j=0
for j in range(numberOfRows):
  for i in action_columns_td3:
   val = action_columns_td3[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_td3, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi4/df_actions_and_account_str_value_td3(4).csv',index=False)
df_actions_str

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,Beli 100.0 saham,Tahan,Beli 100.0 saham,Tahan,200.0,1.000000e+08
1,2018-01-03,Beli 63.0 saham,Tahan,Beli 100.0 saham,Tahan,363.0,9.950120e+07
2,2018-01-04,Tahan,Tahan,Beli 2.0 saham,Tahan,365.0,9.830904e+07
3,2018-01-05,Tahan,Tahan,Tahan,Tahan,365.0,9.904693e+07
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,365.0,1.064543e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,366.0,1.501889e+08
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,366.0,1.504404e+08
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,366.0,1.505661e+08
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,366.0,1.506918e+08


###7.4.1. Agen 4: SAC

In [190]:
#Replace NaN value to 0
action_columns_sac=action_columns_sac.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_sac)

j=0
for j in range(numberOfRows):
  for i in action_columns_sac:
   val = action_columns_sac[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_sac, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi4/df_actions_and_account_str_value_sac(4).csv',index=False)
df_actions_str

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,Tahan,Beli 5.0 saham,Beli 6.0 saham,Beli 100.0 saham,111.0,1.000000e+08
1,2018-01-03,Tahan,Tahan,Tahan,Tahan,111.0,9.643914e+07
2,2018-01-04,Tahan,Tahan,Tahan,Tahan,111.0,9.534190e+07
3,2018-01-05,Tahan,Tahan,Tahan,Tahan,111.0,9.642655e+07
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,111.0,9.714247e+07
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,111.0,7.767142e+07
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,111.0,7.809765e+07
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,111.0,7.749720e+07
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,111.0,7.764024e+07


###7.4.1. Agen 5: DDPG

In [191]:
#Replace NaN value to 0
action_columns_ddpg=action_columns_ddpg.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_ddpg)

j=0
for j in range(numberOfRows):
  for i in action_columns_ddpg:
   val = action_columns_ddpg[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_ddpg, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi4/df_actions_and_account_str_value_ddpg(4).csv',index=False)
df_actions_str

,date,JECC.JK,TMAS.JK,TPIA.JK,UNVR.JK,total_shares,account_value
0,2018-01-02,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,Tahan,300.0,1.000000e+08
1,2018-01-03,Beli 55.0 saham,Beli 100.0 saham,Beli 100.0 saham,Tahan,555.0,9.962652e+07
2,2018-01-04,Jual -100.0 saham,Beli 100.0 saham,Beli 100.0 saham,Tahan,655.0,9.830864e+07
3,2018-01-05,Beli 28.0 saham,Beli 100.0 saham,Beli 100.0 saham,Tahan,883.0,9.923240e+07
4,2018-01-08,Jual -83.0 saham,Beli 100.0 saham,Beli 100.0 saham,Tahan,1000.0,1.033992e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,1262.0,1.645967e+08
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,1262.0,1.655062e+08
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,1262.0,1.658628e+08
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,1262.0,1.662684e+08


<a id='6.4'></a>
##7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi

<a id='6.4.1'></a>
###7.5.1. Agen 1: A2C

In [192]:
#drop some columns
df_actions_and_account_value_a2c = df_actions_and_account_value_a2c.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_a2c:
    df_temp = df_actions_and_account_value_a2c[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic-1:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi4/1_A2C/df_states_and_actions_a2c_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,JECC.JK,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062,67.0
13140,2018-01-03,JECC.JK,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855,67.0
13144,2018-01-04,JECC.JK,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333861,67.0
13148,2018-01-05,JECC.JK,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931331,28.0
13160,2018-01-08,JECC.JK,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,0.0,3.0,-3.619868,53.058014,-40.969126,13.94,3.749016,0.0
15976,2019-12-13,JECC.JK,0.0,4.0,-3.380631,53.058014,-139.322722,12.63,0.593754,0.0
15988,2019-12-16,JECC.JK,0.0,0.0,-3.154669,53.058014,-151.408653,12.14,0.376881,0.0
15992,2019-12-17,JECC.JK,0.0,1.0,-2.941682,53.058014,-167.379002,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,TMAS.JK,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062,0.0
13141,2018-01-03,TMAS.JK,27500.0,2.0,-1.382509,51.234621,-13.715676,9.15,24.801855,0.0
13145,2018-01-04,TMAS.JK,15500.0,3.0,-1.346419,47.766229,-35.329550,9.22,10.333861,0.0
13149,2018-01-05,TMAS.JK,25000.0,4.0,-1.044786,49.533335,-44.898604,9.22,3.931331,0.0
13161,2018-01-08,TMAS.JK,2500.0,0.0,-0.603049,50.824102,28.633446,9.52,8.029031,11.0
...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,3130800.0,3.0,-3.753757,35.769414,-110.875785,13.94,3.749016,0.0
15977,2019-12-13,TMAS.JK,1143700.0,4.0,-3.607022,37.844103,-93.126697,12.63,0.593754,0.0
15989,2019-12-16,TMAS.JK,1726300.0,0.0,-3.515456,37.222220,-89.104445,12.14,0.376881,0.0
15993,2019-12-17,TMAS.JK,2965500.0,1.0,-3.403655,37.222220,-89.887150,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,TPIA.JK,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062,0.0
13142,2018-01-03,TPIA.JK,16574000.0,2.0,18.517531,56.440256,85.907413,9.15,24.801855,0.0
13146,2018-01-04,TPIA.JK,21859600.0,3.0,12.919180,50.876145,-26.782155,9.22,10.333861,0.0
13150,2018-01-05,TPIA.JK,27563200.0,4.0,11.302232,54.003408,1.726900,9.22,3.931331,0.0
13162,2018-01-08,TPIA.JK,25656000.0,0.0,10.878746,54.991439,33.184576,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,32832800.0,3.0,37.659415,62.950311,136.531560,13.94,3.749016,0.0
15978,2019-12-13,TPIA.JK,25854400.0,4.0,40.175702,63.571502,132.220615,12.63,0.593754,0.0
15990,2019-12-16,TPIA.JK,20370000.0,0.0,42.183367,63.884705,123.628289,12.14,0.376881,0.0
15994,2019-12-17,TPIA.JK,20856400.0,1.0,43.769658,64.203090,122.159660,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNVR.JK,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062,0.0
13143,2018-01-03,UNVR.JK,13805500.0,2.0,240.790875,61.141304,83.044372,9.15,24.801855,0.0
13147,2018-01-04,UNVR.JK,9651500.0,3.0,211.040137,58.257807,50.807094,9.22,10.333861,0.0
13151,2018-01-05,UNVR.JK,8882500.0,4.0,193.790704,60.199568,52.000299,9.22,3.931331,0.0
13163,2018-01-08,UNVR.JK,6800000.0,0.0,183.710768,61.436724,70.461991,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,13365500.0,3.0,-49.981812,41.652655,-139.322722,13.94,3.749016,0.0
15979,2019-12-13,UNVR.JK,12891500.0,4.0,-56.350281,42.737188,-151.408653,12.63,0.593754,0.0
15991,2019-12-16,UNVR.JK,12503000.0,0.0,-65.513971,41.582265,-167.379002,12.14,0.376881,0.0
15995,2019-12-17,UNVR.JK,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.29,0.019866,0.0


####String Action Version

In [193]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  #i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi4/1_A2C/df_states_and_actions_str_a2c_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,JECC.JK
0,13136,2018-01-02,JECC.JK,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062,67.0,Beli 67.0 saham
1,13140,2018-01-03,JECC.JK,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855,67.0,Beli 67.0 saham
2,13144,2018-01-04,JECC.JK,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333861,67.0,Beli 67.0 saham
3,13148,2018-01-05,JECC.JK,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931331,28.0,Beli 28.0 saham
4,13160,2018-01-08,JECC.JK,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029031,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,JECC.JK,0.0,3.0,-3.619868,53.058014,-40.969126,13.94,3.749016,0.0,Tahan
491,15976,2019-12-13,JECC.JK,0.0,4.0,-3.380631,53.058014,-139.322722,12.63,0.593754,0.0,Tahan
492,15988,2019-12-16,JECC.JK,0.0,0.0,-3.154669,53.058014,-151.408653,12.14,0.376881,0.0,Tahan
493,15992,2019-12-17,JECC.JK,0.0,1.0,-2.941682,53.058014,-167.379002,12.29,0.019866,0.0,Tahan


,index,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,TMAS.JK
0,13137,2018-01-02,TMAS.JK,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062,0.0,Tahan
1,13141,2018-01-03,TMAS.JK,27500.0,2.0,-1.382509,51.234621,-13.715676,9.15,24.801855,0.0,Tahan
2,13145,2018-01-04,TMAS.JK,15500.0,3.0,-1.346419,47.766229,-35.329550,9.22,10.333861,0.0,Tahan
3,13149,2018-01-05,TMAS.JK,25000.0,4.0,-1.044786,49.533335,-44.898604,9.22,3.931331,0.0,Tahan
4,13161,2018-01-08,TMAS.JK,2500.0,0.0,-0.603049,50.824102,28.633446,9.52,8.029031,11.0,Beli 11.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,TMAS.JK,3130800.0,3.0,-3.753757,35.769414,-110.875785,13.94,3.749016,0.0,Tahan
491,15977,2019-12-13,TMAS.JK,1143700.0,4.0,-3.607022,37.844103,-93.126697,12.63,0.593754,0.0,Tahan
492,15989,2019-12-16,TMAS.JK,1726300.0,0.0,-3.515456,37.222220,-89.104445,12.14,0.376881,0.0,Tahan
493,15993,2019-12-17,TMAS.JK,2965500.0,1.0,-3.403655,37.222220,-89.887150,12.29,0.019866,0.0,Tahan


,index,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,TPIA.JK
0,13138,2018-01-02,TPIA.JK,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062,0.0,Tahan
1,13142,2018-01-03,TPIA.JK,16574000.0,2.0,18.517531,56.440256,85.907413,9.15,24.801855,0.0,Tahan
2,13146,2018-01-04,TPIA.JK,21859600.0,3.0,12.919180,50.876145,-26.782155,9.22,10.333861,0.0,Tahan
3,13150,2018-01-05,TPIA.JK,27563200.0,4.0,11.302232,54.003408,1.726900,9.22,3.931331,0.0,Tahan
4,13162,2018-01-08,TPIA.JK,25656000.0,0.0,10.878746,54.991439,33.184576,9.52,8.029031,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,TPIA.JK,32832800.0,3.0,37.659415,62.950311,136.531560,13.94,3.749016,0.0,Tahan
491,15978,2019-12-13,TPIA.JK,25854400.0,4.0,40.175702,63.571502,132.220615,12.63,0.593754,0.0,Tahan
492,15990,2019-12-16,TPIA.JK,20370000.0,0.0,42.183367,63.884705,123.628289,12.14,0.376881,0.0,Tahan
493,15994,2019-12-17,TPIA.JK,20856400.0,1.0,43.769658,64.203090,122.159660,12.29,0.019866,0.0,Tahan


,index,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,UNVR.JK
0,13139,2018-01-02,UNVR.JK,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062,0.0,Tahan
1,13143,2018-01-03,UNVR.JK,13805500.0,2.0,240.790875,61.141304,83.044372,9.15,24.801855,0.0,Tahan
2,13147,2018-01-04,UNVR.JK,9651500.0,3.0,211.040137,58.257807,50.807094,9.22,10.333861,0.0,Tahan
3,13151,2018-01-05,UNVR.JK,8882500.0,4.0,193.790704,60.199568,52.000299,9.22,3.931331,0.0,Tahan
4,13163,2018-01-08,UNVR.JK,6800000.0,0.0,183.710768,61.436724,70.461991,9.52,8.029031,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,UNVR.JK,13365500.0,3.0,-49.981812,41.652655,-139.322722,13.94,3.749016,0.0,Tahan
491,15979,2019-12-13,UNVR.JK,12891500.0,4.0,-56.350281,42.737188,-151.408653,12.63,0.593754,0.0,Tahan
492,15991,2019-12-16,UNVR.JK,12503000.0,0.0,-65.513971,41.582265,-167.379002,12.14,0.376881,0.0,Tahan
493,15995,2019-12-17,UNVR.JK,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.29,0.019866,0.0,Tahan


<a id='6.4.2'></a>
###7.5.2.Agen 2: PPO

In [194]:
#drop some columns
df_actions_and_account_value_ppo = df_actions_and_account_value_ppo.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_ppo:
    df_temp = df_actions_and_account_value_ppo[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi4/2_PPO/df_states_and_actions_ppo_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,JECC.JK,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062,0.0
13140,2018-01-03,JECC.JK,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855,0.0
13144,2018-01-04,JECC.JK,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333861,0.0
13148,2018-01-05,JECC.JK,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931331,0.0
13160,2018-01-08,JECC.JK,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,0.0,3.0,-3.619868,53.058014,-40.969126,13.94,3.749016,0.0
15976,2019-12-13,JECC.JK,0.0,4.0,-3.380631,53.058014,-139.322722,12.63,0.593754,0.0
15988,2019-12-16,JECC.JK,0.0,0.0,-3.154669,53.058014,-151.408653,12.14,0.376881,0.0
15992,2019-12-17,JECC.JK,0.0,1.0,-2.941682,53.058014,-167.379002,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,TMAS.JK,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062,0.0
13141,2018-01-03,TMAS.JK,27500.0,2.0,-1.382509,51.234621,-13.715676,9.15,24.801855,0.0
13145,2018-01-04,TMAS.JK,15500.0,3.0,-1.346419,47.766229,-35.329550,9.22,10.333861,0.0
13149,2018-01-05,TMAS.JK,25000.0,4.0,-1.044786,49.533335,-44.898604,9.22,3.931331,0.0
13161,2018-01-08,TMAS.JK,2500.0,0.0,-0.603049,50.824102,28.633446,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,3130800.0,3.0,-3.753757,35.769414,-110.875785,13.94,3.749016,0.0
15977,2019-12-13,TMAS.JK,1143700.0,4.0,-3.607022,37.844103,-93.126697,12.63,0.593754,0.0
15989,2019-12-16,TMAS.JK,1726300.0,0.0,-3.515456,37.222220,-89.104445,12.14,0.376881,0.0
15993,2019-12-17,TMAS.JK,2965500.0,1.0,-3.403655,37.222220,-89.887150,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,TPIA.JK,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062,8.0
13142,2018-01-03,TPIA.JK,16574000.0,2.0,18.517531,56.440256,85.907413,9.15,24.801855,8.0
13146,2018-01-04,TPIA.JK,21859600.0,3.0,12.919180,50.876145,-26.782155,9.22,10.333861,8.0
13150,2018-01-05,TPIA.JK,27563200.0,4.0,11.302232,54.003408,1.726900,9.22,3.931331,8.0
13162,2018-01-08,TPIA.JK,25656000.0,0.0,10.878746,54.991439,33.184576,9.52,8.029031,8.0
...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,32832800.0,3.0,37.659415,62.950311,136.531560,13.94,3.749016,0.0
15978,2019-12-13,TPIA.JK,25854400.0,4.0,40.175702,63.571502,132.220615,12.63,0.593754,0.0
15990,2019-12-16,TPIA.JK,20370000.0,0.0,42.183367,63.884705,123.628289,12.14,0.376881,0.0
15994,2019-12-17,TPIA.JK,20856400.0,1.0,43.769658,64.203090,122.159660,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNVR.JK,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062,4.0
13143,2018-01-03,UNVR.JK,13805500.0,2.0,240.790875,61.141304,83.044372,9.15,24.801855,4.0
13147,2018-01-04,UNVR.JK,9651500.0,3.0,211.040137,58.257807,50.807094,9.22,10.333861,4.0
13151,2018-01-05,UNVR.JK,8882500.0,4.0,193.790704,60.199568,52.000299,9.22,3.931331,4.0
13163,2018-01-08,UNVR.JK,6800000.0,0.0,183.710768,61.436724,70.461991,9.52,8.029031,4.0
...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,13365500.0,3.0,-49.981812,41.652655,-139.322722,13.94,3.749016,0.0
15979,2019-12-13,UNVR.JK,12891500.0,4.0,-56.350281,42.737188,-151.408653,12.63,0.593754,0.0
15991,2019-12-16,UNVR.JK,12503000.0,0.0,-65.513971,41.582265,-167.379002,12.14,0.376881,0.0
15995,2019-12-17,UNVR.JK,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.29,0.019866,0.0


####String Action Version

In [195]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi4/2_PPO/df_states_and_actions_str_ppo_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,JECC.JK
0,13136,2018-01-02,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062,Tahan
1,13140,2018-01-03,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855,Tahan
2,13144,2018-01-04,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333861,Tahan
3,13148,2018-01-05,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931331,Tahan
4,13160,2018-01-08,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,0.0,3.0,-3.619868,53.058014,-40.969126,13.94,3.749016,Tahan
491,15976,2019-12-13,0.0,4.0,-3.380631,53.058014,-139.322722,12.63,0.593754,Tahan
492,15988,2019-12-16,0.0,0.0,-3.154669,53.058014,-151.408653,12.14,0.376881,Tahan
493,15992,2019-12-17,0.0,1.0,-2.941682,53.058014,-167.379002,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,TMAS.JK
0,13137,2018-01-02,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062,Tahan
1,13141,2018-01-03,27500.0,2.0,-1.382509,51.234621,-13.715676,9.15,24.801855,Tahan
2,13145,2018-01-04,15500.0,3.0,-1.346419,47.766229,-35.329550,9.22,10.333861,Tahan
3,13149,2018-01-05,25000.0,4.0,-1.044786,49.533335,-44.898604,9.22,3.931331,Tahan
4,13161,2018-01-08,2500.0,0.0,-0.603049,50.824102,28.633446,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,3130800.0,3.0,-3.753757,35.769414,-110.875785,13.94,3.749016,Tahan
491,15977,2019-12-13,1143700.0,4.0,-3.607022,37.844103,-93.126697,12.63,0.593754,Tahan
492,15989,2019-12-16,1726300.0,0.0,-3.515456,37.222220,-89.104445,12.14,0.376881,Tahan
493,15993,2019-12-17,2965500.0,1.0,-3.403655,37.222220,-89.887150,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,TPIA.JK
0,13138,2018-01-02,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062,Beli 8.0 saham
1,13142,2018-01-03,16574000.0,2.0,18.517531,56.440256,85.907413,9.15,24.801855,Beli 8.0 saham
2,13146,2018-01-04,21859600.0,3.0,12.919180,50.876145,-26.782155,9.22,10.333861,Beli 8.0 saham
3,13150,2018-01-05,27563200.0,4.0,11.302232,54.003408,1.726900,9.22,3.931331,Beli 8.0 saham
4,13162,2018-01-08,25656000.0,0.0,10.878746,54.991439,33.184576,9.52,8.029031,Beli 8.0 saham
...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,32832800.0,3.0,37.659415,62.950311,136.531560,13.94,3.749016,Tahan
491,15978,2019-12-13,25854400.0,4.0,40.175702,63.571502,132.220615,12.63,0.593754,Tahan
492,15990,2019-12-16,20370000.0,0.0,42.183367,63.884705,123.628289,12.14,0.376881,Tahan
493,15994,2019-12-17,20856400.0,1.0,43.769658,64.203090,122.159660,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNVR.JK
0,13139,2018-01-02,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062,Beli 4.0 saham
1,13143,2018-01-03,13805500.0,2.0,240.790875,61.141304,83.044372,9.15,24.801855,Beli 4.0 saham
2,13147,2018-01-04,9651500.0,3.0,211.040137,58.257807,50.807094,9.22,10.333861,Beli 4.0 saham
3,13151,2018-01-05,8882500.0,4.0,193.790704,60.199568,52.000299,9.22,3.931331,Beli 4.0 saham
4,13163,2018-01-08,6800000.0,0.0,183.710768,61.436724,70.461991,9.52,8.029031,Beli 4.0 saham
...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,13365500.0,3.0,-49.981812,41.652655,-139.322722,13.94,3.749016,Tahan
491,15979,2019-12-13,12891500.0,4.0,-56.350281,42.737188,-151.408653,12.63,0.593754,Tahan
492,15991,2019-12-16,12503000.0,0.0,-65.513971,41.582265,-167.379002,12.14,0.376881,Tahan
493,15995,2019-12-17,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.29,0.019866,Tahan


<a id='6.4.3'></a>
###7.5.3. Agen 3: TD3

In [196]:
#drop some columns
df_actions_and_account_value_td3 = df_actions_and_account_value_td3.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_td3:
    df_temp = df_actions_and_account_value_td3[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi4/3_TD3/df_states_and_actions_td3_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,JECC.JK,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062,100.0
13140,2018-01-03,JECC.JK,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855,63.0
13144,2018-01-04,JECC.JK,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333861,0.0
13148,2018-01-05,JECC.JK,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931331,0.0
13160,2018-01-08,JECC.JK,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,0.0,3.0,-3.619868,53.058014,-40.969126,13.94,3.749016,0.0
15976,2019-12-13,JECC.JK,0.0,4.0,-3.380631,53.058014,-139.322722,12.63,0.593754,0.0
15988,2019-12-16,JECC.JK,0.0,0.0,-3.154669,53.058014,-151.408653,12.14,0.376881,0.0
15992,2019-12-17,JECC.JK,0.0,1.0,-2.941682,53.058014,-167.379002,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,TMAS.JK,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062,0.0
13141,2018-01-03,TMAS.JK,27500.0,2.0,-1.382509,51.234621,-13.715676,9.15,24.801855,0.0
13145,2018-01-04,TMAS.JK,15500.0,3.0,-1.346419,47.766229,-35.329550,9.22,10.333861,0.0
13149,2018-01-05,TMAS.JK,25000.0,4.0,-1.044786,49.533335,-44.898604,9.22,3.931331,0.0
13161,2018-01-08,TMAS.JK,2500.0,0.0,-0.603049,50.824102,28.633446,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,3130800.0,3.0,-3.753757,35.769414,-110.875785,13.94,3.749016,0.0
15977,2019-12-13,TMAS.JK,1143700.0,4.0,-3.607022,37.844103,-93.126697,12.63,0.593754,0.0
15989,2019-12-16,TMAS.JK,1726300.0,0.0,-3.515456,37.222220,-89.104445,12.14,0.376881,0.0
15993,2019-12-17,TMAS.JK,2965500.0,1.0,-3.403655,37.222220,-89.887150,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,TPIA.JK,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062,100.0
13142,2018-01-03,TPIA.JK,16574000.0,2.0,18.517531,56.440256,85.907413,9.15,24.801855,100.0
13146,2018-01-04,TPIA.JK,21859600.0,3.0,12.919180,50.876145,-26.782155,9.22,10.333861,2.0
13150,2018-01-05,TPIA.JK,27563200.0,4.0,11.302232,54.003408,1.726900,9.22,3.931331,0.0
13162,2018-01-08,TPIA.JK,25656000.0,0.0,10.878746,54.991439,33.184576,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,32832800.0,3.0,37.659415,62.950311,136.531560,13.94,3.749016,0.0
15978,2019-12-13,TPIA.JK,25854400.0,4.0,40.175702,63.571502,132.220615,12.63,0.593754,0.0
15990,2019-12-16,TPIA.JK,20370000.0,0.0,42.183367,63.884705,123.628289,12.14,0.376881,0.0
15994,2019-12-17,TPIA.JK,20856400.0,1.0,43.769658,64.203090,122.159660,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNVR.JK,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062,0.0
13143,2018-01-03,UNVR.JK,13805500.0,2.0,240.790875,61.141304,83.044372,9.15,24.801855,0.0
13147,2018-01-04,UNVR.JK,9651500.0,3.0,211.040137,58.257807,50.807094,9.22,10.333861,0.0
13151,2018-01-05,UNVR.JK,8882500.0,4.0,193.790704,60.199568,52.000299,9.22,3.931331,0.0
13163,2018-01-08,UNVR.JK,6800000.0,0.0,183.710768,61.436724,70.461991,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,13365500.0,3.0,-49.981812,41.652655,-139.322722,13.94,3.749016,0.0
15979,2019-12-13,UNVR.JK,12891500.0,4.0,-56.350281,42.737188,-151.408653,12.63,0.593754,0.0
15991,2019-12-16,UNVR.JK,12503000.0,0.0,-65.513971,41.582265,-167.379002,12.14,0.376881,0.0
15995,2019-12-17,UNVR.JK,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.29,0.019866,0.0


####String Action Version

In [197]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi4/3_TD3/df_states_and_actions_str_td3_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,JECC.JK
0,13136,2018-01-02,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062,Beli 100.0 saham
1,13140,2018-01-03,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855,Beli 63.0 saham
2,13144,2018-01-04,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333861,Tahan
3,13148,2018-01-05,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931331,Tahan
4,13160,2018-01-08,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,0.0,3.0,-3.619868,53.058014,-40.969126,13.94,3.749016,Tahan
491,15976,2019-12-13,0.0,4.0,-3.380631,53.058014,-139.322722,12.63,0.593754,Tahan
492,15988,2019-12-16,0.0,0.0,-3.154669,53.058014,-151.408653,12.14,0.376881,Tahan
493,15992,2019-12-17,0.0,1.0,-2.941682,53.058014,-167.379002,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,TMAS.JK
0,13137,2018-01-02,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062,Tahan
1,13141,2018-01-03,27500.0,2.0,-1.382509,51.234621,-13.715676,9.15,24.801855,Tahan
2,13145,2018-01-04,15500.0,3.0,-1.346419,47.766229,-35.329550,9.22,10.333861,Tahan
3,13149,2018-01-05,25000.0,4.0,-1.044786,49.533335,-44.898604,9.22,3.931331,Tahan
4,13161,2018-01-08,2500.0,0.0,-0.603049,50.824102,28.633446,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,3130800.0,3.0,-3.753757,35.769414,-110.875785,13.94,3.749016,Tahan
491,15977,2019-12-13,1143700.0,4.0,-3.607022,37.844103,-93.126697,12.63,0.593754,Tahan
492,15989,2019-12-16,1726300.0,0.0,-3.515456,37.222220,-89.104445,12.14,0.376881,Tahan
493,15993,2019-12-17,2965500.0,1.0,-3.403655,37.222220,-89.887150,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,TPIA.JK
0,13138,2018-01-02,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062,Beli 100.0 saham
1,13142,2018-01-03,16574000.0,2.0,18.517531,56.440256,85.907413,9.15,24.801855,Beli 100.0 saham
2,13146,2018-01-04,21859600.0,3.0,12.919180,50.876145,-26.782155,9.22,10.333861,Beli 2.0 saham
3,13150,2018-01-05,27563200.0,4.0,11.302232,54.003408,1.726900,9.22,3.931331,Tahan
4,13162,2018-01-08,25656000.0,0.0,10.878746,54.991439,33.184576,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,32832800.0,3.0,37.659415,62.950311,136.531560,13.94,3.749016,Tahan
491,15978,2019-12-13,25854400.0,4.0,40.175702,63.571502,132.220615,12.63,0.593754,Tahan
492,15990,2019-12-16,20370000.0,0.0,42.183367,63.884705,123.628289,12.14,0.376881,Tahan
493,15994,2019-12-17,20856400.0,1.0,43.769658,64.203090,122.159660,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNVR.JK
0,13139,2018-01-02,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062,Tahan
1,13143,2018-01-03,13805500.0,2.0,240.790875,61.141304,83.044372,9.15,24.801855,Tahan
2,13147,2018-01-04,9651500.0,3.0,211.040137,58.257807,50.807094,9.22,10.333861,Tahan
3,13151,2018-01-05,8882500.0,4.0,193.790704,60.199568,52.000299,9.22,3.931331,Tahan
4,13163,2018-01-08,6800000.0,0.0,183.710768,61.436724,70.461991,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,13365500.0,3.0,-49.981812,41.652655,-139.322722,13.94,3.749016,Tahan
491,15979,2019-12-13,12891500.0,4.0,-56.350281,42.737188,-151.408653,12.63,0.593754,Tahan
492,15991,2019-12-16,12503000.0,0.0,-65.513971,41.582265,-167.379002,12.14,0.376881,Tahan
493,15995,2019-12-17,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.29,0.019866,Tahan


<a id='6.4.4'></a>
###7.5.4. Agen 4: SAC

In [198]:
#drop some columns
df_actions_and_account_value_sac = df_actions_and_account_value_sac.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_sac:
    df_temp = df_actions_and_account_value_sac[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi4/4_SAC/df_states_and_actions_sac_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,JECC.JK,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062,0.0
13140,2018-01-03,JECC.JK,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855,0.0
13144,2018-01-04,JECC.JK,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333861,0.0
13148,2018-01-05,JECC.JK,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931331,0.0
13160,2018-01-08,JECC.JK,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,0.0,3.0,-3.619868,53.058014,-40.969126,13.94,3.749016,0.0
15976,2019-12-13,JECC.JK,0.0,4.0,-3.380631,53.058014,-139.322722,12.63,0.593754,0.0
15988,2019-12-16,JECC.JK,0.0,0.0,-3.154669,53.058014,-151.408653,12.14,0.376881,0.0
15992,2019-12-17,JECC.JK,0.0,1.0,-2.941682,53.058014,-167.379002,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,TMAS.JK,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062,5.0
13141,2018-01-03,TMAS.JK,27500.0,2.0,-1.382509,51.234621,-13.715676,9.15,24.801855,0.0
13145,2018-01-04,TMAS.JK,15500.0,3.0,-1.346419,47.766229,-35.329550,9.22,10.333861,0.0
13149,2018-01-05,TMAS.JK,25000.0,4.0,-1.044786,49.533335,-44.898604,9.22,3.931331,0.0
13161,2018-01-08,TMAS.JK,2500.0,0.0,-0.603049,50.824102,28.633446,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,3130800.0,3.0,-3.753757,35.769414,-110.875785,13.94,3.749016,0.0
15977,2019-12-13,TMAS.JK,1143700.0,4.0,-3.607022,37.844103,-93.126697,12.63,0.593754,0.0
15989,2019-12-16,TMAS.JK,1726300.0,0.0,-3.515456,37.222220,-89.104445,12.14,0.376881,0.0
15993,2019-12-17,TMAS.JK,2965500.0,1.0,-3.403655,37.222220,-89.887150,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,TPIA.JK,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062,6.0
13142,2018-01-03,TPIA.JK,16574000.0,2.0,18.517531,56.440256,85.907413,9.15,24.801855,0.0
13146,2018-01-04,TPIA.JK,21859600.0,3.0,12.919180,50.876145,-26.782155,9.22,10.333861,0.0
13150,2018-01-05,TPIA.JK,27563200.0,4.0,11.302232,54.003408,1.726900,9.22,3.931331,0.0
13162,2018-01-08,TPIA.JK,25656000.0,0.0,10.878746,54.991439,33.184576,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,32832800.0,3.0,37.659415,62.950311,136.531560,13.94,3.749016,0.0
15978,2019-12-13,TPIA.JK,25854400.0,4.0,40.175702,63.571502,132.220615,12.63,0.593754,0.0
15990,2019-12-16,TPIA.JK,20370000.0,0.0,42.183367,63.884705,123.628289,12.14,0.376881,0.0
15994,2019-12-17,TPIA.JK,20856400.0,1.0,43.769658,64.203090,122.159660,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNVR.JK,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062,100.0
13143,2018-01-03,UNVR.JK,13805500.0,2.0,240.790875,61.141304,83.044372,9.15,24.801855,0.0
13147,2018-01-04,UNVR.JK,9651500.0,3.0,211.040137,58.257807,50.807094,9.22,10.333861,0.0
13151,2018-01-05,UNVR.JK,8882500.0,4.0,193.790704,60.199568,52.000299,9.22,3.931331,0.0
13163,2018-01-08,UNVR.JK,6800000.0,0.0,183.710768,61.436724,70.461991,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,13365500.0,3.0,-49.981812,41.652655,-139.322722,13.94,3.749016,0.0
15979,2019-12-13,UNVR.JK,12891500.0,4.0,-56.350281,42.737188,-151.408653,12.63,0.593754,0.0
15991,2019-12-16,UNVR.JK,12503000.0,0.0,-65.513971,41.582265,-167.379002,12.14,0.376881,0.0
15995,2019-12-17,UNVR.JK,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.29,0.019866,0.0


####String Action Version

In [199]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi4/4_SAC/df_states_and_actions_str_sac_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,JECC.JK
0,13136,2018-01-02,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062,Tahan
1,13140,2018-01-03,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855,Tahan
2,13144,2018-01-04,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333861,Tahan
3,13148,2018-01-05,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931331,Tahan
4,13160,2018-01-08,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,0.0,3.0,-3.619868,53.058014,-40.969126,13.94,3.749016,Tahan
491,15976,2019-12-13,0.0,4.0,-3.380631,53.058014,-139.322722,12.63,0.593754,Tahan
492,15988,2019-12-16,0.0,0.0,-3.154669,53.058014,-151.408653,12.14,0.376881,Tahan
493,15992,2019-12-17,0.0,1.0,-2.941682,53.058014,-167.379002,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,TMAS.JK
0,13137,2018-01-02,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062,Beli 5.0 saham
1,13141,2018-01-03,27500.0,2.0,-1.382509,51.234621,-13.715676,9.15,24.801855,Tahan
2,13145,2018-01-04,15500.0,3.0,-1.346419,47.766229,-35.329550,9.22,10.333861,Tahan
3,13149,2018-01-05,25000.0,4.0,-1.044786,49.533335,-44.898604,9.22,3.931331,Tahan
4,13161,2018-01-08,2500.0,0.0,-0.603049,50.824102,28.633446,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,3130800.0,3.0,-3.753757,35.769414,-110.875785,13.94,3.749016,Tahan
491,15977,2019-12-13,1143700.0,4.0,-3.607022,37.844103,-93.126697,12.63,0.593754,Tahan
492,15989,2019-12-16,1726300.0,0.0,-3.515456,37.222220,-89.104445,12.14,0.376881,Tahan
493,15993,2019-12-17,2965500.0,1.0,-3.403655,37.222220,-89.887150,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,TPIA.JK
0,13138,2018-01-02,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062,Beli 6.0 saham
1,13142,2018-01-03,16574000.0,2.0,18.517531,56.440256,85.907413,9.15,24.801855,Tahan
2,13146,2018-01-04,21859600.0,3.0,12.919180,50.876145,-26.782155,9.22,10.333861,Tahan
3,13150,2018-01-05,27563200.0,4.0,11.302232,54.003408,1.726900,9.22,3.931331,Tahan
4,13162,2018-01-08,25656000.0,0.0,10.878746,54.991439,33.184576,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,32832800.0,3.0,37.659415,62.950311,136.531560,13.94,3.749016,Tahan
491,15978,2019-12-13,25854400.0,4.0,40.175702,63.571502,132.220615,12.63,0.593754,Tahan
492,15990,2019-12-16,20370000.0,0.0,42.183367,63.884705,123.628289,12.14,0.376881,Tahan
493,15994,2019-12-17,20856400.0,1.0,43.769658,64.203090,122.159660,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNVR.JK
0,13139,2018-01-02,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062,Beli 100.0 saham
1,13143,2018-01-03,13805500.0,2.0,240.790875,61.141304,83.044372,9.15,24.801855,Tahan
2,13147,2018-01-04,9651500.0,3.0,211.040137,58.257807,50.807094,9.22,10.333861,Tahan
3,13151,2018-01-05,8882500.0,4.0,193.790704,60.199568,52.000299,9.22,3.931331,Tahan
4,13163,2018-01-08,6800000.0,0.0,183.710768,61.436724,70.461991,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,13365500.0,3.0,-49.981812,41.652655,-139.322722,13.94,3.749016,Tahan
491,15979,2019-12-13,12891500.0,4.0,-56.350281,42.737188,-151.408653,12.63,0.593754,Tahan
492,15991,2019-12-16,12503000.0,0.0,-65.513971,41.582265,-167.379002,12.14,0.376881,Tahan
493,15995,2019-12-17,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.29,0.019866,Tahan


<a id='6.5.5'></a>
###7.4.5. Agen 5: DDPG

In [200]:
#drop some columns
df_actions_and_account_value_ddpg = df_actions_and_account_value_ddpg.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_ddpg:
    df_temp = df_actions_and_account_value_ddpg[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi4/5_DDPG/df_states_and_actions_ddpg_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,JECC.JK,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062,100.0
13140,2018-01-03,JECC.JK,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855,55.0
13144,2018-01-04,JECC.JK,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333861,-100.0
13148,2018-01-05,JECC.JK,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931331,28.0
13160,2018-01-08,JECC.JK,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029031,-83.0
...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,JECC.JK,0.0,3.0,-3.619868,53.058014,-40.969126,13.94,3.749016,0.0
15976,2019-12-13,JECC.JK,0.0,4.0,-3.380631,53.058014,-139.322722,12.63,0.593754,0.0
15988,2019-12-16,JECC.JK,0.0,0.0,-3.154669,53.058014,-151.408653,12.14,0.376881,0.0
15992,2019-12-17,JECC.JK,0.0,1.0,-2.941682,53.058014,-167.379002,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,TMAS.JK,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062,100.0
13141,2018-01-03,TMAS.JK,27500.0,2.0,-1.382509,51.234621,-13.715676,9.15,24.801855,100.0
13145,2018-01-04,TMAS.JK,15500.0,3.0,-1.346419,47.766229,-35.329550,9.22,10.333861,100.0
13149,2018-01-05,TMAS.JK,25000.0,4.0,-1.044786,49.533335,-44.898604,9.22,3.931331,100.0
13161,2018-01-08,TMAS.JK,2500.0,0.0,-0.603049,50.824102,28.633446,9.52,8.029031,100.0
...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,TMAS.JK,3130800.0,3.0,-3.753757,35.769414,-110.875785,13.94,3.749016,0.0
15977,2019-12-13,TMAS.JK,1143700.0,4.0,-3.607022,37.844103,-93.126697,12.63,0.593754,0.0
15989,2019-12-16,TMAS.JK,1726300.0,0.0,-3.515456,37.222220,-89.104445,12.14,0.376881,0.0
15993,2019-12-17,TMAS.JK,2965500.0,1.0,-3.403655,37.222220,-89.887150,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,TPIA.JK,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062,100.0
13142,2018-01-03,TPIA.JK,16574000.0,2.0,18.517531,56.440256,85.907413,9.15,24.801855,100.0
13146,2018-01-04,TPIA.JK,21859600.0,3.0,12.919180,50.876145,-26.782155,9.22,10.333861,100.0
13150,2018-01-05,TPIA.JK,27563200.0,4.0,11.302232,54.003408,1.726900,9.22,3.931331,100.0
13162,2018-01-08,TPIA.JK,25656000.0,0.0,10.878746,54.991439,33.184576,9.52,8.029031,100.0
...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,TPIA.JK,32832800.0,3.0,37.659415,62.950311,136.531560,13.94,3.749016,0.0
15978,2019-12-13,TPIA.JK,25854400.0,4.0,40.175702,63.571502,132.220615,12.63,0.593754,0.0
15990,2019-12-16,TPIA.JK,20370000.0,0.0,42.183367,63.884705,123.628289,12.14,0.376881,0.0
15994,2019-12-17,TPIA.JK,20856400.0,1.0,43.769658,64.203090,122.159660,12.29,0.019866,0.0


,date,tic,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,UNVR.JK,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062,0.0
13143,2018-01-03,UNVR.JK,13805500.0,2.0,240.790875,61.141304,83.044372,9.15,24.801855,0.0
13147,2018-01-04,UNVR.JK,9651500.0,3.0,211.040137,58.257807,50.807094,9.22,10.333861,0.0
13151,2018-01-05,UNVR.JK,8882500.0,4.0,193.790704,60.199568,52.000299,9.22,3.931331,0.0
13163,2018-01-08,UNVR.JK,6800000.0,0.0,183.710768,61.436724,70.461991,9.52,8.029031,0.0
...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,UNVR.JK,13365500.0,3.0,-49.981812,41.652655,-139.322722,13.94,3.749016,0.0
15979,2019-12-13,UNVR.JK,12891500.0,4.0,-56.350281,42.737188,-151.408653,12.63,0.593754,0.0
15991,2019-12-16,UNVR.JK,12503000.0,0.0,-65.513971,41.582265,-167.379002,12.14,0.376881,0.0
15995,2019-12-17,UNVR.JK,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.29,0.019866,0.0


####String Action Version

In [201]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi4/5_DDPG/df_states_and_actions_str_ddpg_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,JECC.JK
0,13136,2018-01-02,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604062,Beli 100.0 saham
1,13140,2018-01-03,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801855,Beli 55.0 saham
2,13144,2018-01-04,0.0,3.0,-36.877672,43.469840,-55.338547,9.22,10.333861,Tahan
3,13148,2018-01-05,100.0,4.0,-34.841705,43.469840,-53.229574,9.22,3.931331,Beli 28.0 saham
4,13160,2018-01-08,1100.0,0.0,2.197301,64.180298,185.001220,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,0.0,3.0,-3.619868,53.058014,-40.969126,13.94,3.749016,Tahan
491,15976,2019-12-13,0.0,4.0,-3.380631,53.058014,-139.322722,12.63,0.593754,Tahan
492,15988,2019-12-16,0.0,0.0,-3.154669,53.058014,-151.408653,12.14,0.376881,Tahan
493,15992,2019-12-17,0.0,1.0,-2.941682,53.058014,-167.379002,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,TMAS.JK
0,13137,2018-01-02,16500.0,1.0,-2.067100,43.270809,-110.403832,9.77,6.604062,Beli 100.0 saham
1,13141,2018-01-03,27500.0,2.0,-1.382509,51.234621,-13.715676,9.15,24.801855,Beli 100.0 saham
2,13145,2018-01-04,15500.0,3.0,-1.346419,47.766229,-35.329550,9.22,10.333861,Beli 100.0 saham
3,13149,2018-01-05,25000.0,4.0,-1.044786,49.533335,-44.898604,9.22,3.931331,Beli 100.0 saham
4,13161,2018-01-08,2500.0,0.0,-0.603049,50.824102,28.633446,9.52,8.029031,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,3130800.0,3.0,-3.753757,35.769414,-110.875785,13.94,3.749016,Tahan
491,15977,2019-12-13,1143700.0,4.0,-3.607022,37.844103,-93.126697,12.63,0.593754,Tahan
492,15989,2019-12-16,1726300.0,0.0,-3.515456,37.222220,-89.104445,12.14,0.376881,Tahan
493,15993,2019-12-17,2965500.0,1.0,-3.403655,37.222220,-89.887150,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,TPIA.JK
0,13138,2018-01-02,22387200.0,1.0,19.810037,60.719815,142.844593,9.77,6.604062,Beli 100.0 saham
1,13142,2018-01-03,16574000.0,2.0,18.517531,56.440256,85.907413,9.15,24.801855,Beli 100.0 saham
2,13146,2018-01-04,21859600.0,3.0,12.919180,50.876145,-26.782155,9.22,10.333861,Beli 100.0 saham
3,13150,2018-01-05,27563200.0,4.0,11.302232,54.003408,1.726900,9.22,3.931331,Beli 100.0 saham
4,13162,2018-01-08,25656000.0,0.0,10.878746,54.991439,33.184576,9.52,8.029031,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,32832800.0,3.0,37.659415,62.950311,136.531560,13.94,3.749016,Tahan
491,15978,2019-12-13,25854400.0,4.0,40.175702,63.571502,132.220615,12.63,0.593754,Tahan
492,15990,2019-12-16,20370000.0,0.0,42.183367,63.884705,123.628289,12.14,0.376881,Tahan
493,15994,2019-12-17,20856400.0,1.0,43.769658,64.203090,122.159660,12.29,0.019866,Tahan


,index,date,volume,day,macd,rsi_30,cci_30,vix,turbulence,UNVR.JK
0,13139,2018-01-02,12108500.0,1.0,264.457359,71.890177,158.326617,9.77,6.604062,Tahan
1,13143,2018-01-03,13805500.0,2.0,240.790875,61.141304,83.044372,9.15,24.801855,Tahan
2,13147,2018-01-04,9651500.0,3.0,211.040137,58.257807,50.807094,9.22,10.333861,Tahan
3,13151,2018-01-05,8882500.0,4.0,193.790704,60.199568,52.000299,9.22,3.931331,Tahan
4,13163,2018-01-08,6800000.0,0.0,183.710768,61.436724,70.461991,9.52,8.029031,Tahan
...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,13365500.0,3.0,-49.981812,41.652655,-139.322722,13.94,3.749016,Tahan
491,15979,2019-12-13,12891500.0,4.0,-56.350281,42.737188,-151.408653,12.63,0.593754,Tahan
492,15991,2019-12-16,12503000.0,0.0,-65.513971,41.582265,-167.379002,12.14,0.376881,Tahan
493,15995,2019-12-17,10837000.0,1.0,-70.835468,41.956713,-164.670433,12.29,0.019866,Tahan


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.1 Agen A2C

In [202]:
print("===========Get Backtest Results of A2C Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_a2c)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_a2c_"+now+'.csv')

===========Get Backtest Results of A2C Model===========
Annual return          0.193318
Cumulative returns     0.415048
Annual volatility      0.446348
Sharpe ratio           0.614211
Calmar ratio           0.515239
Stability              0.237589
Max drawdown          -0.375201
Omega ratio            1.405098
Sortino ratio          1.051376
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.020311
Daily value at risk   -0.055147
dtype: float64


###8.1.2 Agen ppo

In [203]:
print("===========Get Backtest Results of PPO Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ppo_"+now+'.csv')

===========Get Backtest Results of PPO Model===========
Annual return         -0.006619
Cumulative returns    -0.012960
Annual volatility      0.218870
Sharpe ratio           0.078689
Calmar ratio          -0.024059
Stability              0.023340
Max drawdown          -0.275118
Omega ratio            1.014074
Sortino ratio          0.113788
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.056485
Daily value at risk   -0.027507
dtype: float64


###8.1.3 Agen TD3

In [204]:
print("===========Get Backtest Results of TD3 Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_td3)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_td3_"+now+'.csv')

===========Get Backtest Results of TD3 Model===========
Annual return          0.232674
Cumulative returns     0.508175
Annual volatility      0.344204
Sharpe ratio           0.778238
Calmar ratio           0.776818
Stability              0.492218
Max drawdown          -0.299522
Omega ratio            1.309705
Sortino ratio          1.299552
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.027441
Daily value at risk   -0.042303
dtype: float64


###8.1.4 Agen SAC

In [205]:
print("===========Get Backtest Results of SAC Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_sac)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_sac_"+now+'.csv')

===========Get Backtest Results of SAC Model===========
Annual return         -0.121400
Cumulative returns    -0.224485
Annual volatility      0.251315
Sharpe ratio          -0.390573
Calmar ratio          -0.421647
Stability              0.131372
Max drawdown          -0.287918
Omega ratio            0.932848
Sortino ratio         -0.550884
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.999960
Daily value at risk   -0.032052
dtype: float64


###8.1.5 Agen DDPG

In [215]:
print("===========Get Backtest Results of DDPG Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ddpg)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ddpg_"+now+'.csv')

===========Get Backtest Results of DDPG Model===========
Annual return          0.297227
Cumulative returns     0.667231
Annual volatility      0.332605
Sharpe ratio           0.948255
Calmar ratio           0.919577
Stability              0.226384
Max drawdown          -0.323222
Omega ratio            1.198879
Sortino ratio          1.491324
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.148148
Daily value at risk   -0.040653
dtype: float64
